# A quick tour of DWave Ocean

In this first notebook we introduce some of the main tools in the Ocean library for programming the Dwave quantum computer. We start with a (very) brief discussion of quantum annealing then move on to introduce two important modules of the Ocean library: the dimod and minorminer modules. At the end of the notebook, useful references that dive deeper in the topics discussed here are presented. 

## Quantum Annealing

The idea behind Quantum Annealing is to map an optimization problem to a Hamiltonian on an Ising lattice of the form

\begin{equation}
H(\mathbf{\sigma}) = -\sum J_{ij}\sigma_{i}\sigma_{j} - \sum h_i\sigma_i, \quad \sigma_i = \pm 1.
\end{equation}

Finding the minimum energy of a general Ising lattice Hamiltonian such as the one above consists in determining the set of $\sigma_i$s that minimize the Hamiltonian $H$. Thus problem is known to be [NP-hard](https://en.wikipedia.org/wiki/NP-hardness). In the mathematical optimization community, discrete unconstrained optimization problems formulated in terms of Ising Hamiltonians are known as Quadratic Unconstrained Optimization problems, or [QUBOs](https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization).

Mapping a real world problem into an Ising lattice to be optimized in a DWave computer poses two main theoretical challenges:

- 1. Finding the proper $J$ matrix and $\mathbf{h}$ vector (Energy program);
- 2. Connecting the qubits in the DWave hardware to produce $J$ and $\mathbf{h}$ (Minor embedding).

These two steps appear in the general workflow of quantum adiabatic computation dpicted in the nice picture from the paper <a href= "https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0208561"><i>Open source software in quantum computing</i></a> by Fingerhuth, Babej and Wittek.

<img src="dwave_workflow.png" >




In this first notebook we shall not be very interested in the nasty details of the theory behind quantum annealers as much as in getting our hands dirty with the Python [Ocean](https://docs.ocean.dwavesys.com) library. In particular we shall focus on two key components of the Ocean suite: the `dimod` and the `minorminer` modules.

## The dimod module

The [dimod](https://docs.ocean.dwavesys.com/projects/dimod/en/latest/index.html) module is the Ocean API for modelling binary quadratic optimization problems. It supports both Ising optimization (where $\sigma_i = \pm 1$) and QUBOs (where $q_i = 0,1$). Let us start with the following QUBO.

\begin{equation}
E(\mathbf{a},B;\mathbf{q}) = \mathbf{a}\cdot\mathbf{q} + B\mathbf{q} = -q_0 - q_1 + 2q_0q_1 
\end{equation}

where we have 
\begin{equation}
\mathbf{a} = (a_0,a_1) = (-1,-1)
\end{equation}
and 
\begin{equation}
B = \begin{bmatrix}
B_{00} & B_{01}\\
B_{10} & B_{11}
\end{bmatrix} = \begin{bmatrix}
0 & 2\\
0 & 0
\end{bmatrix}
\end{equation}

We want to find the binary vector $\mathbf{q} = (q_0,q_1)$ that minimizes $E$. Let us do this with dimod.

In [36]:
import dimod
qubo = dimod.BinaryQuadraticModel({0: -1, 1: -1}, {(0, 1): 2}, 0.0, dimod.BINARY)  
qubo_exact_response = dimod.ExactSolver().sample(qubo)
for sample, energy in qubo_exact_response.data(['sample', 'energy']): 
    print("sample: ", sample, "\t","energy: ", energy)

sample:  {0: 1, 1: 0} 	 energy:  -1.0
sample:  {0: 0, 1: 1} 	 energy:  -1.0
sample:  {0: 0, 1: 0} 	 energy:  0.0
sample:  {0: 1, 1: 1} 	 energy:  0.0


In the code snippet above, we have used the `ExactSolver` sampler to find the energy by brute force of all possible combinations. Evidently, this will quickly become infeasible as the dimensionality of the QUBO grows. So let us investigate what other possible samplers we have. We can use Python's introspection tools to find a little bit more about the `dimod` model. Bellow we print the methods, attributes and functions contained in `dimod`.

In [15]:
", ".join(dir(dimod))

'BINARY, BQM, BinaryPolynomial, BinaryQuadraticModel, BinaryQuadraticModelSizeError, BinaryQuadraticModelStructureError, BinaryQuadraticModelValueError, ComposedPolySampler, ComposedSampler, Composite, ExactSolver, FixedVariableComposite, HigherOrderComposite, InvalidComposition, InvalidSampler, MappingError, NullSampler, PolySampler, PolyScaleComposite, PolyTruncateComposite, RandomSampler, Response, RoofDualityComposite, SPIN, SampleSet, Sampler, ScaleComposite, SimulatedAnnealingSampler, SpinReversalTransformComposite, StructureComposite, Structured, TrackingComposite, TruncateComposite, Vartype, WriteableError, _, __author__, __authoremail__, __builtins__, __cached__, __description__, __doc__, __file__, __loader__, __name__, __package__, __path__, __spec__, __version__, absolute_import, as_samples, as_vartype, binary_quadratic_model, bqm_structured, child_structure_dfs, compatibility23, composite, composites, concatenate, core, decorators, dimod, exact_solver, exceptions, fix_varia

From the list above, we see that several classes like `RandomSampler` and `SimulatedAnnealingSampler` are available for sampling. We can skip Google for a while and again use Python's introspection capabilities to learn a little about the `SimulatedAnnealingSampler` class.

In [19]:
help(dimod.SimulatedAnnealingSampler)

Help on class SimulatedAnnealingSampler in module dimod.reference.samplers.simulated_annealing:

class SimulatedAnnealingSampler(dimod.core.sampler.Sampler)
 |  A simple simulated annealing sampler for testing and debugging code.
 |  
 |  Examples:
 |      This example solves a two-variable Ising model.
 |  
 |      >>> h = {'a': -0.5, 'b': 1.0}
 |      >>> J = {('a', 'b'): -1.5}
 |      >>> sampleset = dimod.SimulatedAnnealingSampler().sample_ising(h, J)
 |  
 |  Method resolution order:
 |      SimulatedAnnealingSampler
 |      dimod.core.sampler.Sampler
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  sample(self, bqm, beta_range=None, num_reads=10, num_sweeps=1000)
 |      Sample from low-energy spin states using simulated annealing.
 |      
 |      Args:
 |          bqm (:obj:`.BinaryQuadraticModel`):
 |              Binary quadratic model to be sampled from.
 |      
 | 

This is cool, the `SimulatedAnnealingSampler` contains a method `sample` that accepts parameters such as the inverse temperature $\beta$. Let us run this sampler.

In [34]:
qubo_simulated_annealing_response = dimod.SimulatedAnnealingSampler().sample(qubo)
for sample, energy in qubo_simulated_annealing_response.data(['sample', 'energy']): 
    print("sample: ", sample, "energy: ", energy)

sample:  {0: 1, 1: 0} energy:  -1.0
sample:  {0: 0, 1: 1} energy:  -1.0
sample:  {0: 0, 1: 1} energy:  -1.0
sample:  {0: 1, 1: 0} energy:  -1.0
sample:  {0: 1, 1: 0} energy:  -1.0
sample:  {0: 0, 1: 1} energy:  -1.0
sample:  {0: 0, 1: 1} energy:  -1.0
sample:  {0: 0, 1: 1} energy:  -1.0
sample:  {0: 1, 1: 1} energy:  0.0
sample:  {0: 1, 1: 1} energy:  0.0


Note that we got 10 samples, since that is the default value of the `num_reads` parameter. 

Now let us solve the same quadratic optimization problem in the Ising formulation. The QUBO is easily written as an Ising Hamiltonian via the transformation

\begin{equation}
\sigma_i = 2q_i - 1,
\end{equation}

so the corresponding Ising problem is 

\begin{equation}
H(\mathbf{h},\text{J};\mathbf{\sigma}) = -\mathbf{h}\cdot\mathbf{\sigma} - J\mathbf{\sigma} = \frac{1}{2}(\sigma_0\sigma_1 - 1)
\end{equation}

so we have

\begin{equation}
\mathbf{h} = (h_0, h_1) = (0,0)
\end{equation}

and 

\begin{equation}
J = \begin{bmatrix}
J_{00} & J_{01}\\
J_{10} & J_{11}
\end{bmatrix} = \begin{bmatrix}
0 & \frac{1}{2}\\
0 & 0
\end{bmatrix}
\end{equation},

with an offset term of $-\frac{1}{2}$. The code snippet below solves this Ising problem exactly.




In [39]:
ising = dimod.BinaryQuadraticModel({0: 0, 1: 0}, {(0, 1): 0.5}, -0.5, dimod.SPIN)  
ising_exact_response = dimod.ExactSolver().sample(ising)
for sample, energy in ising_exact_response.data(['sample', 'energy']): 
    print("sample: ", sample, "\t", "energy: ", energy)

sample:  {0: 1, 1: -1} 	 energy:  -1.0
sample:  {0: -1, 1: 1} 	 energy:  -1.0
sample:  {0: -1, 1: -1} 	 energy:  0.0
sample:  {0: 1, 1: 1} 	 energy:  0.0


## The minorminer module

In [2]:
import dwave_networkx as dnx

In [ ]:
[minorminer](https://docs.ocean.dwavesys.com/projects/minorminer/en/latest/index.html)